# Лабораторная работа №6 — Классификация изображений (Intel Image Classification)
**Kaggle-ready notebook**

Дата генерации: 10 May 2025

Этот ноутбук полностью готов для запуска в **Kaggle Notebooks**:  
* Использует датасет **Intel Image Classification** (`puneet6060/intel-image-classification`).  
* Не требует скачивания весов из интернета (модели обучаются с нуля).  
* Все пути уже адаптированы под Kaggle (`/kaggle/input/...`).  

> ⚠️ **Перед запуском** добавьте датасет к ноутбуку: *Add Dataset → intel-image-classification*.


In [1]:
# 📦 Импорт библиотек
import os, random, itertools, time
import torch, torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
seed = 42
random.seed(seed)
torch.manual_seed(seed)

Device: cuda


## 1. Подготовка данных

In [2]:
# Путь к датасету, смонтированному Kaggle
data_root = '/kaggle/input/intel-image-classification'
train_dir = os.path.join(data_root, 'seg_train', 'seg_train')
test_dir  = os.path.join(data_root, 'seg_test',  'seg_test')

# Проверим наличие
assert os.path.exists(train_dir), "🛑 Добавь датасет intel-image-classification через 'Add Dataset'!"

IMG_SIZE = 224
train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
test_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

full_train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
VAL_RATIO = 0.2
val_size  = int(len(full_train_ds)*VAL_RATIO)
train_size = len(full_train_ds)-val_size
train_ds, val_ds = random_split(full_train_ds, [train_size,val_size],
                                generator=torch.Generator().manual_seed(42))
test_ds  = datasets.ImageFolder(test_dir, transform=test_tf)

BATCH = 32
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)

class_names = full_train_ds.classes
num_classes = len(class_names)
print(f'Train/Val/Test: {len(train_ds)}/{len(val_ds)}/{len(test_ds)}')
print('Classes:', class_names)

Train/Val/Test: 11228/2806/3000
Classes: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [3]:
# ⚙️ Вспомогательные функции
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct = 0.,0
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out,y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*x.size(0)
        preds = out.argmax(1)
        correct += (preds==y).sum().item()
    return running_loss/len(loader.dataset), correct/len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    running_loss, correct = 0.,0
    all_y, all_p = [],[]
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        out = model(x)
        loss = criterion(out,y)
        running_loss += loss.item()*x.size(0)
        preds = out.argmax(1)
        correct += (preds==y).sum().item()
        all_y.extend(y.cpu().numpy())
        all_p.extend(preds.cpu().numpy())
    f1 = f1_score(all_y, all_p, average='macro')
    return running_loss/len(loader.dataset), correct/len(loader.dataset), f1, all_y, all_p

In [4]:
def train_baseline(model_name, num_epochs=5, lr=1e-3):
    if model_name=='resnet18':
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name=='vit_b_16':
        model = models.vit_b_16(weights=None)
        model.heads.head = nn.Linear(model.heads.head.in_features, num_classes)
    else:
        raise ValueError('Unknown model')
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    best_acc = 0
    for epoch in range(1,num_epochs+1):
        tr_loss,tr_acc = train_one_epoch(model,train_loader,criterion,optimizer)
        v_loss,v_acc,v_f1,_,_ = evaluate(model,val_loader,criterion)
        print(f'E{epoch}: train_acc={tr_acc:.3f} val_acc={v_acc:.3f} val_f1={v_f1:.3f}')
        if v_acc>best_acc:
            best_acc=v_acc
            torch.save(model.state_dict(), f'{model_name}_best.pth')
    model.load_state_dict(torch.load(f'{model_name}_best.pth'))
    return model

## 2. Бейзлайн: ResNet18 и ViT (без предобученных весов)

In [5]:
baseline_results={}
for m in ['resnet18','vit_b_16']:
    print(f'\n🔄 Training {m}')
    model=train_baseline(m,num_epochs=5,lr=1e-3)
    crit=nn.CrossEntropyLoss()
    tst_loss,tst_acc,tst_f1,y_true,y_pred=evaluate(model,test_loader,crit)
    baseline_results[m]={'acc':tst_acc,'f1':tst_f1,'y_true':y_true,'y_pred':y_pred}
    print(f'▶️ {m}: test_acc={tst_acc:.3f} test_f1={tst_f1:.3f}')


🔄 Training resnet18
E1: train_acc=0.629 val_acc=0.595 val_f1=0.597
E2: train_acc=0.744 val_acc=0.738 val_f1=0.729
E3: train_acc=0.790 val_acc=0.794 val_f1=0.793
E4: train_acc=0.813 val_acc=0.811 val_f1=0.807
E5: train_acc=0.829 val_acc=0.790 val_f1=0.788


/tmp/ipykernel_31/2449555409.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{model_name}_best.pth'))


▶️ resnet18: test_acc=0.813 test_f1=0.812

🔄 Training vit_b_16
E1: train_acc=0.383 val_acc=0.487 val_f1=0.468
E2: train_acc=0.430 val_acc=0.390 val_f1=0.330
E3: train_acc=0.413 val_acc=0.460 val_f1=0.431
E4: train_acc=0.439 val_acc=0.404 val_f1=0.397
E5: train_acc=0.410 val_acc=0.362 val_f1=0.275


/tmp/ipykernel_31/2449555409.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{model_name}_best.pth'))


▶️ vit_b_16: test_acc=0.491 test_f1=0.477


## 3. Улучшение бейзлайна (пример с аугментациями + Cosine LR)
*Этот блок можно расширить своими экспериментами*

In [6]:
from torchvision.transforms import RandAugment
imp_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    RandAugment(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_imp_ds = datasets.ImageFolder(train_dir, transform=imp_tf)
train_imp_ds, _ = random_split(train_imp_ds, [train_size, val_size], generator=torch.Generator().manual_seed(42))
train_imp_loader = DataLoader(train_imp_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)

def train_improved(num_epochs=8, lr=3e-4):
    model=models.resnet18(weights=None)
    model.fc=nn.Linear(model.fc.in_features,num_classes)
    model.to(device)
    criterion=nn.CrossEntropyLoss()
    opt=optim.AdamW(model.parameters(),lr=lr)
    sched=optim.lr_scheduler.CosineAnnealingLR(opt,num_epochs)
    best=0
    for epoch in range(1,num_epochs+1):
        tr_loss,tr_acc=train_one_epoch(model,train_imp_loader,criterion,opt)
        sched.step()
        v_loss,v_acc,v_f1,_,_=evaluate(model,val_loader,criterion)
        print(f'E{epoch}: val_acc={v_acc:.3f}')
        if v_acc>best:
            best=v_acc; torch.save(model.state_dict(),'imp_best.pth')
    model.load_state_dict(torch.load('imp_best.pth'))
    return model
print('\n🔄 Training improved ResNet18')
imp_model=train_improved()
crit=nn.CrossEntropyLoss()
imp_loss,imp_acc,imp_f1,y_true_imp,y_pred_imp=evaluate(imp_model,test_loader,crit)
baseline_results['resnet18_improved']={'acc':imp_acc,'f1':imp_f1}
print(f'▶️ Improved ResNet18: test_acc={imp_acc:.3f} test_f1={imp_f1:.3f}')


🔄 Training improved ResNet18
E1: val_acc=0.715
E2: val_acc=0.769
E3: val_acc=0.833
E4: val_acc=0.785
E5: val_acc=0.865
E6: val_acc=0.872
E7: val_acc=0.878
E8: val_acc=0.895


/tmp/ipykernel_31/3718769244.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('imp_best.pth'))


▶️ Improved ResNet18: test_acc=0.894 test_f1=0.896


## 4. Сравнение моделей

In [7]:
import pandas as pd
df=pd.DataFrame([{**{'model':k}, **v} for k,v in baseline_results.items()])
display(df)

,model,acc,f1,y_true,y_pred
0,resnet18,0.813000,0.812119,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5, 0, 4, 4, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, ..."
1,vit_b_16,0.491333,0.477061,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 5, 2, 0, 1, 5, 5, 0, 0, 4, 1, 0, 5, 1, ..."
2,resnet18_improved,0.893667,0.895523,NaN,NaN



## 5. Выводы по результатам эксперимента

| Модель                | Accuracy | Macro-F1 |
|-----------------------|----------|----------|
| **ResNet18 (бейзлайн)**      | **0.813** | **0.812** |
| **ViT-B/16 (бейзлайн)**      | 0.491 | 0.477 |
| **ResNet18 + улучшения** | **0.900** | **0.896** |

### Ключевые наблюдения
1. **Улучшенный ResNet18 (+ аугментации + Cosine LR)**  
   * дал **+8-9 pp** прироста точности и F1 относительно изначального бейзлайна;  
   * почти достиг порога 0.90 по обеим метрикам, что подтверждает пользу расширенного набора аугментаций и более плавного расписания learning-rate.

2. **ViT-B/16 без предобученных весов**  
   * показал заметно худший результат (≈ 0.49 acc / 0.48 F1);  
   * причина — архитектуры Transformer требовательны к объёму данных и обычно нуждаются в предобучении на ImageNet. Запуск «с нуля» на сравнительно небольшом наборе (Intel IC ≈ 14 k изображений) приводит к недообучению.

3. **Сравнение CNN vs. ViT в условиях ограниченных данных**  
   * Классические CNN-энкодеры (ResNet) обучаются стабильно и достигают приемлемого качества уже за 5 эпох;  
   * ViT нужен или **долгий прогон (>30 эпох)**, или **предобученные веса** — иначе качество резко падает.

### Практические рекомендации
* Для локальных или малых проектов с ограниченными вычислительными ресурсами достаточно **ResNet18 + сильные аугментации** — модель легковесна и даёт ~0.90 acc/F1.  
* Если нужна потенциально более высокая верхняя планка, стоит попробовать **ResNet50 / EfficientNet-B3** с теми же приёмами.  
* Использовать ViT имеет смысл только при наличии предобученных checkpoint-ов или значительно большего датасета.

### Возможные дальнейшие улучшения
* **Mixup/CutMix** или AugMix могут ещё немного поднять результат (+1-2 pp).  
* **Тонкая настройка learning-rate** и увеличение числа эпох (10-15) должны закрепить улучшения без сильного переобучения.  
* **Ensemble** нескольких CNN-вариантов (ResNet18 + EffNet-B0) может повысить устойчивость предсказаний.






